In [1]:
# Step 0: Install dependencies (run once)
!pip install -q transformers datasets evaluate seqeval sklearn shap lime

# Step 1: Upload your .conll dataset file
from google.colab import files

print("Please upload your Amharic NER dataset (.conll file):")
uploaded = files.upload()

# The uploaded file name
filename = list(uploaded.keys())[0]
print(f"Uploaded file: {filename}")

# Step 2: Read your .conll file into tokens and tags
def read_conll(filename):
    tokens = []
    ner_tags = []
    all_tokens = []
    all_tags = []
    with open(filename, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                if tokens:
                    all_tokens.append(tokens)
                    all_tags.append(ner_tags)
                    tokens = []
                    ner_tags = []
                continue
            splits = line.split()
            if len(splits) < 2:
                continue
            token, tag = splits[0], splits[-1]
            tokens.append(token)
            ner_tags.append(tag)
    # Add last sentence if file doesn't end with newline
    if tokens:
        all_tokens.append(tokens)
        all_tags.append(ner_tags)
    return all_tokens, all_tags

tokens, ner_tags = read_conll(filename)

# Step 3: Create label mappings
unique_tags = list(set(tag for doc in ner_tags for tag in doc))
unique_tags.sort()

tag2id = {tag: i for i, tag in enumerate(unique_tags)}
id2tag = {i: tag for tag, i in tag2id.items()}

print(f"Labels: {unique_tags}")

# Convert ner tags to ids
ner_tags_ids = [[tag2id[t] for t in seq] for seq in ner_tags]

# Step 4: Create HuggingFace Dataset and split into train/test
from datasets import Dataset, DatasetDict

data_dict = {"tokens": tokens, "ner_tags": ner_tags_ids}
dataset = Dataset.from_dict(data_dict) # Create dataset directly from parsed data

split = dataset.train_test_split(test_size=0.2, seed=42)
dataset = DatasetDict({"train": split["train"], "test": split["test"]})

# Step 5: Tokenize and align labels function
from transformers import AutoTokenizer

def tokenize_and_align_labels(examples, tokenizer, label_all_tokens=True):
    tokenized_inputs = tokenizer(
        examples["tokens"], is_split_into_words=True, truncation=True, padding="max_length", max_length=128
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Step 6: Setup models to fine-tune
model_names = {
    "xlm-roberta": "xlm-roberta-base",
    "distilbert": "distilbert-base-multilingual-cased",
    "mbert": "bert-base-multilingual-cased",
}

# Step 7: Setup metric for evaluation
import numpy as np
# from datasets import load_metric # Removed deprecated import
import evaluate # Import evaluate library

# metric = load_metric("seqeval") # Removed deprecated function call
metric = evaluate.load("seqeval") # Use evaluate.load()

def compute_metrics(p):
    predictions, labels = p
    preds = np.argmax(predictions, axis=2)

    true_labels = [[id2tag[l] for l in label if l != -100] for label in labels]
    true_preds = [
        [id2tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, labels)
    ]

    results = metric.compute(predictions=true_preds, references=true_labels) # Changed references to true_labels
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Step 8: Fine-tune, evaluate and compare all models
from transformers import AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification # Import DataCollator

best_f1 = 0
best_model_id = None

for model_id, model_name in model_names.items():
    print(f"\n\n\n🧠 Fine-tuning: {model_id} ({model_name})")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

    tokenized_datasets = dataset.map(
        lambda examples: tokenize_and_align_labels(examples, tokenizer),
        batched=True,
    )

    # Initialize DataCollatorForTokenClassification
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)


    training_args = TrainingArguments(
        output_dir=f"./results_{model_id}",
        eval_strategy="epoch", # Changed from evaluation_strategy
        save_strategy="epoch", # Added to match eval_strategy for load_best_model_at_end=True
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_dir=f"./logs_{model_id}",
        logging_steps=10,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        save_total_limit=1,
        seed=42,
        report_to="none", # Disable reporting to services like Weights & Biases
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        tokenizer=tokenizer,
        data_collator=data_collator, # Added data_collator
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_result = trainer.evaluate()
    print(f"Eval result for {model_id}: {eval_result}")

    if eval_result["eval_f1"] > best_f1:
        best_f1 = eval_result["eval_f1"]
        best_model_id = model_id
        # Save best model + tokenizer
        trainer.save_model(f"./best_model_{model_id}")
        tokenizer.save_pretrained(f"./best_model_{model_id}")

print(f"\n\n🏆 Best model: {best_model_id} with F1 = {best_f1:.4f}")

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Please upload your Amharic NER dataset (.conll file):


Saving amharic_ner_data.conll to amharic_ner_data (17).conll
Uploaded file: amharic_ner_data (17).conll
Labels: ['B-LOC', 'B-PRICE', 'B-Product', 'I-LOC', 'I-PRICE', 'I-Product', 'O']


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(





🧠 Fine-tuning: xlm-roberta (xlm-roberta-base)


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

/tmp/ipython-input-1-3773550426.py:162: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.713312,0.000000,0.000000,0.000000,1.000000
2,No log,0.692801,0.000000,0.000000,0.000000,1.000000
3,0.834900,0.680811,0.000000,0.000000,0.000000,1.000000


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:557: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score ar

Eval result for mbert: {'eval_loss': 0.713312029838562, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_accuracy': 1.0, 'eval_runtime': 7.1947, 'eval_samples_per_second': 2.085, 'eval_steps_per_second': 0.139, 'epoch': 3.0}


🏆 Best model: xlm-roberta with F1 = 1.0000


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:557: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [2]:
trainer.save_model("amharic-ner-xlm-roberta")
tokenizer.save_pretrained("amharic-ner-xlm-roberta")


('amharic-ner-xlm-roberta/tokenizer_config.json',
 'amharic-ner-xlm-roberta/special_tokens_map.json',
 'amharic-ner-xlm-roberta/vocab.txt',
 'amharic-ner-xlm-roberta/added_tokens.json',
 'amharic-ner-xlm-roberta/tokenizer.json')